<center>

# What's Cooking Challenge

### <i> Progetto per l'esame di Programmazione di applicazioni di Data Intensive (2019) </i>

### Cichetti Federico, Sponziello Nicolò
</center>

Il progetto ha lo scopo di creare un modello in grado di classificare il tipo di cucina di una ricetta in base agli ingredienti forniti.

## Librerie

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import pickle
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.svm import SVC

%matplotlib inline

nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')
nltk.download("stopwords")

## Esplorazione dei dati

Partiamo caricando i dati in un dataframe Pandas e visualizzandone una parte per capire come sono strutturati

In [ ]:
df = pd.read_json("train.json")
pd.options.display.max_colwidth = 500

In [ ]:
df.head()

In [ ]:
df.tail()

Il dataframe ha tre colonne:
- "cuisine" indica il tipo di cucina a cui appartiene il piatto. Questa sarà l'incognita da scoprire.
- "id" è una colonna che contiene un numero identificativo di ogni piatto. Questo dato non è utile al problema, quindi decidiamo di eliminare la colonna e usare come identificativo l'indice aggiunto in automatico da Pandas.
- "ingredients" contiene la lista di ingredienti del piatto.

In [ ]:
if "id" in df:
    df.drop("id", inplace=True, axis=1)
df.head()

Prima di procedere si esplorano e visualizzano alcuni dati, in particolare:
* Quante ricette sono presenti
* La totalità degli ingredienti

In [ ]:
len(df)

Le ricette da classificare nel dataset sono 39774 in totale.

In [ ]:
len(df['cuisine'].unique())

In totale ci sono 20 tipi di cucine differenti. Si tratta quindi di un problema di classificazione multiclasse.
Controlliamo quanti piatti ci sono per ogni cucina.

In [ ]:
df['cuisine'].value_counts()

In [ ]:
df['cuisine'].value_counts().plot("bar")

Dal grafico possiamo notare che le classi sono sbilanciate, cioè sono presenti molte ricette che vengono identificate come cucina italiana e messicana, mentre ci sono poche ricette russe e brasiliane.

In [ ]:
###################################BILANCIAMENTO???????################################################

Per analizzare gli ingredienti, per adesso usiamo un approccio iterativo. Si crea un set di ingredienti in modo da eliminare eventuali duplicati.

In [ ]:
ingredients = set()
for recipe in df['ingredients']:
    for i in recipe:
        ingredients.add(i)

In [ ]:
len(ingredients)

In totale notiamo che in tutto ci sono 6714 ingredienti diversi nel dataset.

Visualizziamo quelli più usati. Per fare questo gli ingredienti vanno inseriti in una lista in modo da mantenere i duplicati che poi dovranno essere contati.

In [ ]:
ingredients_list = list()
for i in df['ingredients']:
    for word in i:
        ingredients_list.append(word)

In [ ]:
common_ingredients = pd.Series(ingredients_list)
common_ingredients.value_counts().nlargest(15)

Questi sono gli ingredienti più comuni

In [ ]:
common_ingredients.value_counts().nlargest(20).plot(kind="bar")

Visualizziamo ora gli ingredienti più comuni per ogni cucina. Si crea un dizionario che contiene per ogni cucina un dizionario ingrediente -> numero di occorrenze e lo si ordina per tale conteggio. Possiamo poi costruire un DataFrame per visualizzare efficacemente quali sono gli ingredienti più usati.

In [ ]:
tmp = df.groupby('cuisine')['ingredients'].apply(list)

def most_common_ingr_by_cuisine(cuisine):
    lists = tmp[cuisine]
    res = defaultdict(int)
    for recipe in lists:
        for ingr in recipe:
            res[ingr] += 1
    return sorted(res.items(), key=lambda x: x[1], reverse=True)

Chiamando la funzione qui sopra passando, ad esempio, la classe "italian" possiamo vedere che sale, olio di oliva, aglio e parmigiano sono alcuni degli ingredienti più comuni della cucina italiana.

In [ ]:
commons_ital = pd.DataFrame(most_common_ingr_by_cuisine('italian'), columns=["ingredient", "count"]).head(10)
commons_ital

In realtà il sale è l'ingrediente più comune per molte cucine.

In [ ]:
pd.DataFrame(most_common_ingr_by_cuisine('french'), columns=["ingredient", "count"]).head(3)

In [ ]:
pd.DataFrame(most_common_ingr_by_cuisine('brazilian'), columns=["ingredient", "count"]).head(3)

Definiamo ora una funzione che restituisca il numero di ingredienti medio per ogni ricetta di una determinata cucina.

In [ ]:
def avg_ingr_per_recipe(cuisine):
    recipes = tmp[cuisine]
    count = 0;
    for recipe in recipes:
        for ingr in recipe:
            count += 1;
    return count/len(recipes)

avg_ingr_per_recipe('brazilian')

In [ ]:
average = {cuisine: avg_ingr_per_recipe(cuisine) for cuisine in df['cuisine'].unique()}
average

In [ ]:
plt.figure(figsize=(21, 3))
plt.bar(average.keys(), average.values(), align="center", width=0.5)
plt.title("Ingredienti medi per ricetta per ogni cucina")
plt.show()

# Preprocessing dei dati

In questa fase, con i risultati dell'analisi, effettuiamo la trasformazione dei dati per essere pronti per essere elaborati dagli algoritmi di learning.

Come abbiamo potuto notare, il dataframe si compone di righe formate da:

- **[cuisine]**: categoria di cucina
- **[ingredients]**: lista di ingredienti in formato testuale

Come possiamo gestire gli ingredienti di una ricetta?

 - Considerandoli cosi come sono posti, cioè "black olives" rimane "black olives"
 - Unirli in un unica stringa e applicare tecniche di text processing per cercare di estrarre più informazioni
   - Stemming, Lemming, Bag of Word, Vector Space Model (Tfidf)

L'approccio iniziale che abbiamo provato, è stato quello di estrarre un set da tutti gli ingredienti presenti e binarizzare ogni ricetta presente
    - Avremmo ottenuto un dataset con circa 6700 features e 40'000 istanze, con un'alta occupazione di memoria e lunghi calcoli CPU
    - Per ogni riga, che rappresenta un recipe è associato un vettore in [0, 1] in cui la cella corrispondente all'ingrediente contiene
    1 se è usato nella ricetta, 0 altrimenti

Si è rivelato però inefficiente e inaccurato, percui abbiamo scelto un approccio differente

### Tfidf

L'approccio scelto è stato quello di utilizzare il modello tf-idf che si adatta particolarmente bene allo scopo
 - Tfidf associa ad ogni parola un peso che dipende sia dalla frequenza di uso locale Tf (cioè nella stessa ricetta) sia negli altri documenti idf (ricette)
 - Tutti gli ingredienti vengono mappati
 - I valori per ogni parola sono normalizzati in [0, 1]
 

Inoltre permette di:
 - Eliminare eventuali stopword
 - Considerare sia singole parole, che ngram

Per prima cosa, manipoliamo la colonna 'ingredients' del dataframe
 - Da lista di ingredienti la trasformiamo in una unica stringa

In [ ]:
for idx in df.index:
    txt = ""
    for ing in df.loc[idx, "ingredients"]:
        txt += (ing + " ")
    df.loc[idx, "ingredients"] = txt

In [ ]:
df.head()

Creiamo i set per il training dei modelli e il calcolo dello score sul validation

In [ ]:
X_t, X_v, y_t, y_v = train_test_split(df['ingredients'], df['cuisine'], random_state=42, test_size=1/3)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Perceptron

Iniziamo lo studio dei modelli di classificazione dal più semplice, il Perceptron

In [ ]:
perceptron = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("perc", Perceptron())
])

Per ottimizzare i parametri, usiamo la Grid Search
 - Testiamo anche quale ngram ottimizza lo score

In [ ]:
grid1 = {
    'tfidf__ngram_range':[(1, 1), (1, 2), (1, 3), (1, 4)],
    'perc__penalty': ['l1', 'l2']
}
gs_perc = GridSearchCV(perceptron, param_grid=grid, n_jobs=-1)

In [ ]:
gs_perc.fit(X_t, y_t)

In [ ]:
gs_perc.score(X_v, y_v)